# pyTorch Computer Vision

* See Reference notebok- https://github.com/mrdbourke/pytorch-deep-learning/blob/main/03_pytorch_computer_vision.ipynb
* See referance Online book- https://www.learnpytorch.io/03_pytorch_computer_vision/

### 0. Computer vision libaries in PyTorch

* `torchvision` - base domain libary for PyTorch computer vision
* `torchvision.datasets` - get datasets and data loading functions for computer vision here
* `torchvision.models` - get pretrained computer vision models that you can leverage for your own problems
* `torchvision.transforms`- functions for manipulating your vision data (images) to be suitable for use with an ML model
* `torch.utils.data.Dataset` - Base dataset class for PyTorch.
* `torch.utils.data.DataLoader` - Create a Python iterable over a dataset

In [ ]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision 
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
print (torch.__version__)
print(torchvision.__version__)

## Getting a Dataset

The dataset we will be using FashionMNIST torchvision.datasets- https://pytorch.org/vision/0.18/generated/torchvision.datasets.FashionMNIST.html

In [ ]:
# Setup Training Data 
from torchvision import datasets
train_data = datasets.FashionMNIST(
    root="data" , #where to download data to?
    train=True, # do we want the training dataset?
    download = True, #do we want to download yes/no?
    transform=torchvision.transforms.ToTensor(), # How do we want to transfrom the data?
    target_transform=None #How do we want to trnsfrom the labels/targets?
    
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
len (train_data), len(test_data)

In [ ]:
# See first training sample
image, label = train_data[0]
image, label

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
# Check the shape of our image
print (f"image shape:{image.shape} -> [color_chanel, height, width]")
print (f" image label: {class_names[label]}")


In [ ]:
import matplotlib.pyplot as plt
image, label = train_data[0]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze()) # image shape is [1, 28, 28] (colour channels, height, width)
plt.title(label);

In [ ]:
plt.imshow(image.squeeze(), cmap="gray")
plt.title(class_names[label]);
plt.axis(False);

In [ ]:
# Plot more images
#torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows*cols+1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title(class_names[label])
    plt.axis(False)

Do You think these items of clothing (images) could be modelled with pure linear lines? Or do you tghink we'll need non-linearities?

## 2. Prepare DataLoader

Right now, ourdata is in the from of PyTorch Datasets.

DataLoader turns our dataset into a Python iterable.

More Specidfically, we want to turn our data into batches (or mini-batches).

Why would we do this?

1. It is more computationally efficient, as in, Your computing hardware may not be able to look (store in memory) at 6000 images in one hit. So we break it down to 32 images at a time (batch size of 32).

2. It gives our neural network more chances to update its per epoch.

For more on mini-batches , see here: https://www.youtube.com/watch?v=4qJaSmvhxi8

In [ ]:
from torch.utils.data import DataLoader

#setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn dataset into iterables (batches)
train_dataloader = DataLoader (dataset=train_data,
                               batch_size= BATCH_SIZE,
                               shuffle=True)

test_dataloader = DataLoader (dataset=test_data,
                              batch_size= BATCH_SIZE,
                              shuffle = False)

train_dataloader, test_dataloader


In [ ]:
# Lets check out what we've created
print (f"DataLoader: {train_dataloader, test_dataloader}")
print(f"Length of train_dataloader:{len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of test_dataloader:{len(test_dataloader)} batches of {BATCH_SIZE}...")